In [1]:
!pip install yfinance
!pip install arch
!pip install pmdarima

In [ ]:
synth_columns = [
    "Receita",
    "Lucro Bruto",
    "Lucro Líquido",
    "Despesas Operacionais",
    "Margem Bruta",
    "Margem Líquida",
    "EBITDA",
    "Fluxo de Caixa Operacional",
    "Fluxo de Caixa Livre",
    "Retorno sobre o Investimento (ROI)",
    "Retorno sobre o Patrimônio Líquido (ROE)",
    "Endividamento",
    "Liquidez Corrente",
    "Liquidez Rápida",
    "Ponto de Equilíbrio",
    "Crescimento da Receita",
    "Crescimento do Lucro",
    "Participação de Mercado",
    "Taxa de Retenção de Clientes",
    "Custo de Aquisição de Clientes (CAC)",
    "Lifetime Value (LTV) do Cliente",
    "Taxa de Churn",
    "Número de Funcionários",
    "Crescimento de Funcionários",
    "Investimentos em P&D",
    "Capex (Capital Expenditure)",
    "Opex (Operational Expenditure)",
    "Taxa de Conversão de Vendas",
    "Margem de Contribuição",
    "Tempo Médio de Recebimento",
    "Tempo Médio de Pagamento",
    "Ciclo de Caixa",
    "Índice de Satisfação dos Funcionários",
    "Índice de Satisfação dos Clientes",
    "Taxa de Utilização da Capacidade",
    "Investimento em Marketing",
    "Taxa de Retorno de Produtos",
    "Taxa de Crescimento de Novos Clientes",
    "Número de Parcerias Estratégicas",
    "Taxa de Crescimento de Receita por Região",
]

In [109]:
from scipy.stats import norm, gamma, beta

# Definir o intervalo de datas
date_range = pd.date_range(start='2001-01-02', end='2024-12-29', freq='D')

# Criar um DataFrame com a coluna Date
data = pd.DataFrame({'Date': date_range})

def generate_revenue_growth(start_value, end_value, periods, volatility):
    """
    Gera uma série de receita com crescimento log-normal estocástico.

    :param start_value: Valor inicial da receita
    :param end_value: Valor final desejado da receita
    :param periods: Número de períodos
    :param volatility: Volatilidade do processo estocástico
    :return: Série de receitas diárias
    """
    # Calcular a taxa média de crescimento diário
    total_growth = end_value / start_value
    daily_growth = total_growth ** (1 / periods) - 1

    # Gerar retornos diários log-normais
    mu = np.log(1 + daily_growth) - 0.5 * volatility**2
    returns = np.random.lognormal(mu, volatility, periods)

    # Calcular a série de receitas
    revenue_series = start_value * np.cumprod(returns)

    # Ajustar para garantir que o valor final seja próximo ao desejado
    scaling_factor = end_value / revenue_series[-1]
    revenue_series *= scaling_factor

    return revenue_series

# Parâmetros para a geração de receita
start_revenue_annual = 500_000_000  # Receita anual inicial (500 milhões)
end_revenue_annual = 3_000_000_000  # Receita anual final desejada (3 bilhões)
volatility = 0.01  # Volatilidade diária

# Gerar série de receitas diárias
revenue_series = generate_revenue_growth(
    start_revenue_annual / 365,
    end_revenue_annual / 365,
    len(date_range),
    volatility
)

# Adicionar componente sazonal
seasonal_component = 0.2 * np.sin(2 * np.pi * np.arange(len(date_range)) / 365)
revenue_series *= (1 + seasonal_component)

data['Receita'] = revenue_series

# Funções para gerar outras métricas financeiras
def generate_cost_of_goods_sold(revenue):
    return revenue * np.random.beta(8, 12)  # CMV entre 30% e 50% da receita

def generate_operating_expenses(revenue):
    return revenue * np.random.beta(5, 15)  # Despesas operacionais entre 15% e 35% da receita

def generate_ebitda(gross_profit, operating_expenses):
    return gross_profit - operating_expenses

def generate_net_profit(ebitda):
    tax_rate = np.random.beta(7, 13)  # Taxa de imposto entre 25% e 45%
    return ebitda * (1 - tax_rate)

# Gerar outras métricas financeiras
data['CMV'] = data['Receita'].apply(generate_cost_of_goods_sold)
data['Lucro Bruto'] = data['Receita'] - data['CMV']
data['Despesas Operacionais'] = data['Receita'].apply(generate_operating_expenses)
data['EBITDA'] = generate_ebitda(data['Lucro Bruto'], data['Despesas Operacionais'])
data['Lucro Líquido'] = generate_net_profit(data['EBITDA'])

# Calcular métricas derivadas
data['Margem Bruta'] = data['Lucro Bruto'] / data['Receita']
data['Margem EBITDA'] = data['EBITDA'] / data['Receita']
data['Margem Líquida'] = data['Lucro Líquido'] / data['Receita']

# Gerar métricas de mercado e operacionais
data['Participação de Mercado'] = np.random.beta(10, 90, len(date_range))  # Entre 5% e 15%
data['Taxa de Retenção de Clientes'] = np.random.beta(80, 20, len(date_range))  # Entre 70% e 90%
data['CAC'] = gamma.rvs(2, scale=500, size=len(date_range))  # Custo de Aquisição de Clientes
data['LTV'] = data['CAC'] * np.random.uniform(3, 5, len(date_range))  # Lifetime Value do Cliente
data['Taxa de Churn'] = 1 - data['Taxa de Retenção de Clientes']

# Gerar métricas de eficiência e produtividade
initial_employees = 500
max_employees = 5000
employee_growth = np.random.normal(0.0002, 0.0001, len(date_range))  # Crescimento diário médio de 0.02% com variação
data['Número de Funcionários'] = np.cumsum(employee_growth) + 1
data['Número de Funcionários'] = np.floor(initial_employees * np.exp(data['Número de Funcionários']))
data['Número de Funcionários'] = np.minimum(data['Número de Funcionários'], max_employees)
data['Receita por Funcionário'] = data['Receita'] / data['Número de Funcionários']
data['Lucro por Funcionário'] = data['Lucro Líquido'] / data['Número de Funcionários']

# Gerar métricas de investimento e inovação
data['Investimentos em P&D'] = data['Receita'] * np.random.beta(5, 95, len(date_range))  # Entre 1% e 5% da receita
data['Capex'] = data['Receita'] * np.random.beta(10, 90, len(date_range))  # Entre 5% e 15% da receita
data['Nível de Inovação'] = np.cumsum(np.random.normal(0, 0.01, len(date_range))) % 1  # Índice entre 0 e 1

# Gerar métricas financeiras adicionais
data['ROI'] = data['Lucro Líquido'] / (data['Capex'] + data['Investimentos em P&D'])
data['ROE'] = data['Lucro Líquido'] / (data['Receita'] * np.random.beta(20, 80, len(date_range)))  # Patrimônio Líquido estimado
data['Liquidez Corrente'] = np.random.gamma(shape=4, scale=0.5, size=len(date_range))
data['Endividamento'] = data['Receita'] * np.random.beta(10, 90, len(date_range))  # Entre 5% e 15% da receita

# Calcular taxas de crescimento
for metric in ['Receita', 'Lucro Líquido', 'Número de Funcionários']:
    data[f'Crescimento de {metric}'] = data[metric].pct_change()

# Adicionar algumas métricas de eficiência operacional
data['Ciclo de Conversão de Caixa'] = np.random.randint(30, 90, len(date_range))
data['Giro de Estoque'] = np.random.uniform(4, 8, len(date_range))
data['Eficiência Operacional'] = data['Receita'] / data['Despesas Operacionais']

In [110]:
data.dropna()

,Date,Receita,CMV,Lucro Bruto,Despesas Operacionais,EBITDA,Lucro Líquido,Margem Bruta,Margem EBITDA,Margem Líquida,...,ROI,ROE,Liquidez Corrente,Endividamento,Crescimento de Receita,Crescimento de Lucro Líquido,Crescimento de Número de Funcionários,Ciclo de Conversão de Caixa,Giro de Estoque,Eficiência Operacional
1,2001-01-03,8.134876e+06,3.103163e+06,5.031713e+06,1.679731e+06,3.351982e+06,2.183344e+06,0.618536,0.412051,0.268393,...,1.545271,1.438700,0.453774,840872.233783,0.003791,0.118076,0.000000,40,6.138760,4.842963
2,2001-01-04,8.316444e+06,4.200927e+06,4.115518e+06,3.331943e+06,7.835750e+05,5.103886e+05,0.494865,0.094220,0.061371,...,0.342348,0.252063,3.114752,578841.943114,0.022320,-0.766235,0.000000,64,7.084874,2.495975
3,2001-01-05,8.328288e+06,3.661497e+06,4.666791e+06,3.528472e+06,1.138319e+06,7.414542e+05,0.560354,0.136681,0.089028,...,0.773554,0.401050,1.799201,827362.573496,0.001424,0.452725,0.000736,40,6.931791,2.360310
4,2001-01-06,8.316303e+06,2.985093e+06,5.331211e+06,2.798644e+06,2.532567e+06,1.649610e+06,0.641055,0.304530,0.198359,...,1.328319,0.958691,2.263876,684669.134844,-0.001439,1.224831,0.000000,76,7.510651,2.971547
5,2001-01-07,8.347533e+06,3.460023e+06,4.887510e+06,9.419642e+05,3.945546e+06,2.569967e+06,0.585504,0.472660,0.307871,...,1.794660,1.393170,2.295594,893027.845760,0.003755,0.557924,0.000000,46,7.799770,8.861836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8758,2024-12-25,7.700061e+06,3.085768e+06,4.614293e+06,2.097811e+06,2.516482e+06,1.639133e+06,0.599254,0.326813,0.212873,...,1.557808,0.953964,2.673324,917868.049603,0.025255,1.640162,0.000000,52,7.734268,3.670522
8759,2024-12-26,7.866709e+06,3.064894e+06,4.801814e+06,6.664653e+05,4.135349e+06,2.693597e+06,0.610397,0.525677,0.342405,...,2.279468,1.707004,3.124755,599456.902292,0.021642,0.643305,0.000000,48,5.059563,11.803628
8760,2024-12-27,8.038689e+06,4.121806e+06,3.916883e+06,8.610558e+05,3.055827e+06,1.990441e+06,0.487254,0.380140,0.247608,...,1.729188,0.927853,3.577809,789018.644655,0.021862,-0.261047,0.000000,46,4.705256,9.335852
8761,2024-12-28,8.111315e+06,5.155588e+06,2.955727e+06,1.920402e+06,1.035326e+06,6.743686e+05,0.364396,0.127640,0.083139,...,0.615526,0.571308,3.423300,978786.664296,0.009035,-0.661196,0.000000,31,6.406227,4.223759


In [112]:
macro_data = [
    "^DJI",      # Dow Jones Industrial Average
    "^IXIC",     # Nasdaq Composite
    "^BVSP",     # Ibovespa
    "BRL=X",     # Taxa de Câmbio USD/BRL
    "CL=F",      # Petróleo (WTI)
    "GC=F",      # Ouro
    "SI=F",      # Prata
    "^FTSE",     # FTSE 100 (Londres)
    "^N225",     # Nikkei 225 (Japão)
    "^HSI",      # Hang Seng Index (Hong Kong)
    "^DAX",      # DAX (Alemanha)
    "^CAC40",    # CAC 40 (França)
    "^AORD",     # All Ordinaries Index (Austrália)
    "^IBEX",     # IBEX 35 (Espanha)
    "^MXX",      # IPC (México)
    "VIX",       # CBOE Volatility Index (Índice de Volatilidade)
    "TLT",       # iShares 20+ Year Treasury Bond ETF (Títulos de Longo Prazo dos EUA)
    "SPY",       # SPDR S&P 500 ETF Trust (Fundo de Índice S&P 500)
    "XAUUSD=X",  # Preço do Ouro em USD
    "EUR=X",     # Taxa de Câmbio USD/EUR
    "JPY=X",     # Taxa de Câmbio USD/JPY
    "GBP=X",     # Taxa de Câmbio USD/GBP
    "AUD=X",     # Taxa de Câmbio USD/AUD
    "CNY=X",     # Taxa de Câmbio USD/CNY
    "TBF",       # ProShares Short 20+ Year Treasury (Títulos de Longo Prazo dos EUA)
    "BND",       # Vanguard Total Bond Market ETF (Mercado de Títulos dos EUA)
    "HYG",       # iShares iBoxx $ High Yield Corporate Bond ETF (Títulos Corporativos de Alto Rendimento)
    "LQD",       # iShares iBoxx $ Investment Grade Corporate Bond ETF (Títulos Corporativos de Grau de Investimento)
    "UUP",       # Invesco DB US Dollar Index Bullish Fund (Índice do Dólar dos EUA)
    "DXY",       # US Dollar Index (Índice do Dólar dos EUA)
    "JNK",       # SPDR Bloomberg Barclays High Yield Bond ETF (Títulos de Alto Rendimento)
    "IEF",       # iShares 7-10 Year Treasury Bond ETF (Títulos de 7 a 10 anos dos EUA)
    "IEI"        # iShares 3-7 Year Treasury Bond ETF (Títulos de 3 a 7 anos dos EUA)
]

In [114]:
import pandas as pd
import yfinance as yf

df2 = yf.download('^GSPC', start='2001-01-01', end='2025-01-01').reset_index()
df2 = df2[['Date', 'Close']]
df2.rename(columns={'Close': 'Close_GSPC'}, inplace=True)

for i in macro_data:
  try:
      df2_aux = yf.download(i, start='2001-01-01', end='2024-01-01').reset_index()
      df2_aux = df2_aux[['Date', 'Close']]
      df2_aux[f'Close_{i}'] = df2_aux['Close']
      df2_aux.drop(columns=['Close'], inplace=True)

      df2 = df2.merge(df2_aux, on='Date', how='left')
  except:
    pass

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['^DAX']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2001-01-01 -> 2024-01-01)')
[******

In [115]:
df2

,Date,Close_GSPC,Close_^DJI,Close_^IXIC,Close_^BVSP,Close_BRL=X,Close_CL=F,Close_GC=F,Close_SI=F,Close_^FTSE,...,Close_CNY=X,Close_TBF,Close_BND,Close_HYG,Close_LQD,Close_UUP,Close_DXY,Close_JNK,Close_IEF,Close_IEI
0,2001-01-02,1283.270020,10646.150391,2291.860107,15425.0,NaN,27.200001,268.399994,4.545,6174.700195,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001-01-03,1347.560059,10945.750000,2616.689941,16599.0,NaN,27.950001,268.000000,4.490,6039.899902,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2001-01-04,1333.339966,10912.410156,2566.830078,16675.0,NaN,28.200001,267.299988,4.532,6185.600098,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2001-01-05,1298.349976,10662.009766,2407.649902,16410.0,NaN,28.000000,268.000000,4.555,6198.100098,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2001-01-08,1295.859985,10621.349609,2395.919922,16562.0,NaN,27.350000,268.000000,4.545,6149.600098,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5948,2024-08-26,5616.839844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5949,2024-08-27,5625.799805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5950,2024-08-28,5592.180176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5951,2024-08-29,5591.959961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
df2 = df2.copy()[-1000:]

num_nan = df2.isnull().sum()
colunas_filtradas = num_nan[num_nan < 50].index
df_filtrado = df2[colunas_filtradas]

In [117]:
df_filtrado = df_filtrado.dropna()

In [118]:
df_filtrado

,Date,Close_GSPC
4953,2020-09-10,3339.189941
4954,2020-09-11,3340.969971
4955,2020-09-14,3383.540039
4956,2020-09-15,3401.199951
4957,2020-09-16,3385.489990
...,...,...
5948,2024-08-26,5616.839844
5949,2024-08-27,5625.799805
5950,2024-08-28,5592.180176
5951,2024-08-29,5591.959961


In [119]:
df_filtrado = df_filtrado.merge(data, on='Date', how='left')

In [120]:
df_filtrado.reset_index().reset_index().head(3)

,level_0,index,Date,Close_GSPC,Receita,CMV,Lucro Bruto,Despesas Operacionais,EBITDA,Lucro Líquido,...,ROI,ROE,Liquidez Corrente,Endividamento,Crescimento de Receita,Crescimento de Lucro Líquido,Crescimento de Número de Funcionários,Ciclo de Conversão de Caixa,Giro de Estoque,Eficiência Operacional
0,0,0,2020-09-10,3339.189941,1.177176e+07,8.232561e+06,3.539196e+06,1.037900e+06,2.501296e+06,1.629242e+06,...,0.846477,0.962219,3.733187,1.188640e+06,0.008088,-0.085288,0.0,30,4.343685,11.341901
1,1,1,2020-09-11,3340.969971,1.190012e+07,5.648554e+06,6.251570e+06,2.388042e+06,3.863528e+06,2.516544e+06,...,1.894764,0.817321,0.650207,1.756025e+06,0.010905,0.544610,0.0,39,4.354585,4.983215
2,2,2,2020-09-14,3383.540039,1.212492e+07,3.729898e+06,8.395020e+06,3.857424e+06,4.537595e+06,2.955603e+06,...,1.894014,1.246172,3.095084,9.108784e+05,0.009410,-0.033011,0.0,55,7.075502,3.143268
